In [2]:
# TU PHAM
# Questions 1, 2, 4
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "UNKNOWN"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end


# Question 3
function separateIntoParagraphs(essays)
    array = Array{String}[]
    for i in eachindex(essays)
        paragraphs = split(essays[i], "\r\n\r\n")
        for j in eachindex(paragraphs)
            paragraphs[j] = replace(paragraphs[j], "\r\n" => " ")
        end
        push!(array, paragraphs)
    end
    return array
end

array = separateIntoParagraphs(essays)

# Questions 5, 6, 7, 8
function editText(text)
    editedText = lowercase(text)
    editedText = replace(editedText, r"\(\d+\)" => "")
    editedText = replace(editedText, r"\d+" => "")
    editedText = replace(editedText, "\r\n" => " ")
    editedText = replace(editedText, r"[,:;'-]"=> "")
    return editedText
end

function editEssays(essays)
    for i in eachindex(essays)
        essays[i] = editText(essays[i])
    end
    return essays
end

function editParagraphs(array)
    for i in eachindex(array)
        for j in eachindex(array[i])
            array[i][j] = editText(array[i][j])
        end
    end
    return array
end

essays = editEssays(essays)
array = editParagraphs(array)

# Question 9
authorSet = ["HAMILTON", "MADISON", "JAY", "UNKNOWN", "ALL"]

function addParagraphsByAuthor(paragraphs, authors)
    essaysByAuthor = [Array{String}[], Array{String}[], Array{String}[], Array{String}[], Array{String}[]]
    for i in eachindex(authors)
        if(cmp(authors[i], "HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
        elseif(cmp(authors[i], "MADISON") == 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "MADISON, WITH HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "JAY") == 0)
            jay = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(jay, paragraphs[i][j])
            end
            push!(essaysByAuthor[3], jay)
        else
            unknown = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(unknown, paragraphs[i][j])
            end
            push!(essaysByAuthor[4], unknown)
        end
    end
    essaysByAuthor[5] = paragraphs
    return essaysByAuthor
end

paragraphsByAuthor = addParagraphsByAuthor(array, authors)

# Question 9a
function avgWordsPerEssay(authorParagraphs)
    wordCount = 0
    for i in eachindex(authorParagraphs)
        for j in eachindex(authorParagraphs[i])
            wordCount += (count(" ", authorParagraphs[i][j]) + 1)
        end
    end
    return wordCount / length(authorParagraphs)
end
println("Average number of words per essay by Madison is $(avgWordsPerEssay(paragraphsByAuthor[2])), by Hamilton is $(avgWordsPerEssay(paragraphsByAuthor[1])), by Jay is $(avgWordsPerEssay(paragraphsByAuthor[3])), by JAY is $(avgWordsPerEssay(paragraphsByAuthor[4])) and OVERALL is $(avgWordsPerEssay(paragraphsByAuthor[5]))")

function find_essays_by_author(essays, author)
    indices = findall(x -> x == author, authors)
    return essays[indices]
end

function average_word_count(essays)
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        total_word_count += length(words)
    end
    return total_word_count, length(essays)
end

function average_word_length(essays)
    total_word_length = 0
    total_word_count = 0
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            total_word_length += length(word)
        end
        total_word_count += length(words)
    end
    return total_word_length / total_word_count
end

function average_sentence_count(essays)
    total_sentence_count = 0
    for essay in essays
        sentences = split(essay, r"[.?!]")
        total_sentence_count += length(sentences)
    end
    return total_sentence_count / length(essays)
end

function average_sentence_per_paragraph(essays)
    total_sentence_count = 0
    total_paragraph_count = 0
    for essay in essays
        paragraphs = split(essay, "\n")
        total_paragraph_count += length(paragraphs)
        for paragraph in paragraphs
            sentences = split(paragraph, r"[.?!]")
            total_sentence_count += length(sentences)
        end
    end
    return total_sentence_count / total_paragraph_count
end

function unique_word_count(essays)
    word_dict = Dict()
    for essay in essays
        words = split(essay, r"\s+")
        for word in words
            if haskey(word_dict, word)
                word_dict[word] += 1
            else
                word_dict[word] = 1
            end
        end
    end
    return word_dict
end

hamilton_words = Dict()
madison_words = Dict()
jay_words = Dict()
hamilton_words_total = 0
madison_words_total = 0
jay_words_total = 0

for author in authorSet
    author_essays = find_essays_by_author(essays, author) 
    println(author, " wrote ", length(author_essays), " essays.")
    total_word_count,total_essay_count = average_word_count(author_essays)
    println("Average word count per essay: ", (total_word_count/total_essay_count))
    println("Average word length per essay: ", average_word_length(author_essays))
    println("Average sentence count per essay: ", average_sentence_count(author_essays))
    println("Average sentence count per paragraph: ", average_sentence_per_paragraph(author_essays))
    word_dict = unique_word_count(author_essays)
    #obtain the word count dictionaries for each author
    if(cmp(author, "HAMILTON") == 0)
        hamilton_words = word_dict
        hamilton_words_total = total_word_count
    elseif(cmp(author, "MADISON") == 0)
        madison_words = word_dict  
        madison_words_total = total_word_count
    elseif(cmp(author, "JAY") == 0)
        jay_words = word_dict   
        jay_words_total = total_word_count
    end
   # println("Unique words and their frequency:")
    #for (word, count) in word_dict
     #   println(word, ": ", count)
      #  
       # end

end

# Define the score function
# Currently need to write into the loop above to save time
function score(author, word)
    M = get(madison_words, word, 0) / madison_words_total
    H = get(hamilton_words, word, 0) / hamilton_words_total
    J = get(jay_words, word, 0) / jay_words_total
    if(cmp(author, "HAMILTON") == 0)
        return abs(H - M) + abs(H - J)
    elseif (cmp(author, "MADISON") == 0)
        return abs(M - H) + abs(M - J)
    elseif (cmp(author, "JAY") == 0)
        return abs(J - H) + abs(J - M)
    end
end

# Find the 10 words with the highest score for each author
madison_scores = Dict()
for word in keys(madison_words)
    madison_scores[word] = score("MADISON", word)
end
madison_top10 = sort(collect(madison_scores), by=x->x[2], rev=true)[1:10]

hamilton_scores = Dict()
for word in keys(hamilton_words)
    hamilton_scores[word] = score("HAMILTON", word)
end
hamilton_top10 = sort(collect(hamilton_scores), by=x->x[2], rev=true)[1:10]

jay_scores = Dict()
for word in keys(jay_words)
    jay_scores[word] = score("JAY", word)
end
jay_top10 = sort(collect(jay_scores), by=x->x[2], rev=true)[1:10]

# Print the results
println("Top 10 words for Madison:")
for word in madison_top10
    if(cmp(word, madison_top10[end]) == 0)
        println(" $word[1]")
    else
        print(" $word[1],")
    end
end

println("Top 10 words for Hamilton:")
for word in hamilton_top10
    if(cmp(word, hamilton_top10[end]) == 0)
        println(" $word[1]")
    else
        print(" $word[1],")
    end
end

println("Top 10 words for Jay:")
for word in jay_top10
    if(cmp(word, jay_top10[end]) == 0)
        println(" $word[1]")
    else
        print(" $word[1],")
    end
end


Average number of words per essay by Hamilton is 2179.823529411765, by Madison is 2768.5714285714284, by Jay is 1674.2, by Jay is 1976.0 and overall is 2211.0823529411764


Hamilton wrote 0 essays.
Average word count per essay: NaN
Average word length per essay: NaN
Average sentence count per essay: NaN
Average sentence count per paragraph: NaN
Madison wrote 0 essays.
Average word count per essay: NaN
Average word length per essay: NaN
Average sentence count per essay: NaN


Average sentence count per paragraph: NaN
Jay wrote 0 essays.
Average word count per essay: NaN
Average word length per essay: NaN
Average sentence count per essay: NaN
Average sentence count per paragraph: NaN
Unknown wrote 0 essays.
Average word count per essay: NaN
Average word length per essay: NaN
Average sentence count per essay: NaN
Average sentence count per paragraph: NaN
All wrote 0 essays.
Average word count per essay: NaN
Average word length per essay: NaN
Average sentence count per essay: NaN
Average sentence count per paragraph: NaN


BoundsError: BoundsError: attempt to access 0-element Vector{Pair{Any, Any}} at index [1:10]

In [ ]:
# Tester
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
println(length(essays))
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end

# println(essays[1])

In [ ]:
# Minh's implementation

# Note: the number of the essay will be index no.

# Break it up into the individual essays. Each essay starts with "Federalist No. n." Within each essay you can skip everything up to and including "To the People of the State of New York:" and the footnotes after "PUBLIUS." You can also skip everything up to the start of the first essay and everything after the end of the last essay. (3 points)
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)

from = "To the People of the State of New York:"
to = "PUBLIUS" 

# Loop over each essay and extract the text in the essays
for (i, essay) in enumerate(essays)

    search_from = findfirst(from, essay)
    search_to = findfirst(to, essay)

    # Find the indices of from and to in the essay string
    start_index = search_from[1] + length(from)
    end_index = search_to[1] - 1
    
    # Extract the text between from and to
    extracted_text = essay[start_index:end_index]

    # Remove trailing new lines in the start and the end of the needed text
    extracted_text = rstrip(extracted_text) # somehow this does not remove the last line of the last paragraph, but it create a concrete pattern for all paragraphs (to have one new line at the end), so I guess it works.
    extracted_text = lstrip(extracted_text)

    # Print the extracted text
    # println(extracted_text)
    # println(i)

    # Replace with only the needed text from each essay
    essays[i] = extracted_text 
    
end
# println(typeof(essays[1]))
println(essays[1])

